## Configuration

In [14]:
!pip install git_root

PROJECT_ROOT = None
in_colab = 'google.colab' in str(get_ipython())

if in_colab:
  print('Running on CoLab')
  PROJECT_ROOT = "/content/drive/MyDrive/DL4NLP/abstract-to-title-generation"
  from google.colab import drive
  drive.mount('/content/drive')

else:
  print('Running on local machine')
  from git_root import git_root
  PROJECT_ROOT = git_root()

%cd {PROJECT_ROOT}

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Running on CoLab
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/DL4NLP/abstract-to-title-generation


In [1]:
# install requirements
!pip install -r requirements.txt

# pull data only pulls changed data
!dvc pull

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Running on CoLab
Mounted at /content/drive
/content/drive/MyDrive/DL4NLP/abstract-to-title-generation
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 12.8 MB/s 
     |████████████████████████████████| 4.4 MB 40.3 MB/s 
     |████████████████████████████████| 1.2 MB 61.3 MB/s 
     |████████████████████████████████| 401 kB 65.1 MB/s 
     |████████████████████████████████| 1.1 MB 58.0 MB/s 
     |████████████████████████████████| 140 kB 76.8 MB/s 
     |████████████████████████████████| 101 kB 12.4 MB/s 
     |████████████████████████████████| 212 kB 72.2 MB/s 
     |████████████████████████████████| 596 kB 54.5 MB/s 
     |████████████████████████████████| 127 kB 69.8 MB/s 
     |████████████████████████████████| 6.6 MB 49.8 MB/s 
     |████████████████████████████████| 144 kB 61.3 MB/s 
    

Computing md5 for a large file '/content/drive/MyDrive/DL4NLP/abstract-to-title-generation/.dvc/cache/9c/152208403a0dbde1d60f817cf9d6cb'. This is only done once.
 12% 4/32 [00:00<00:02, 12.92files/s{'info': ''}]
  0% 0.00/1.51G [00:00<?, ?B/s]
  0% 0.00/1.51G [00:00<?, ?B/s{'info': ''}]
  0% 1.00M/1.51G [00:00<08:59, 3.01MB/s{'info': ''}]
  1% 11.0M/1.51G [00:00<00:50, 32.2MB/s{'info': ''}]
  2% 30.0M/1.51G [00:00<00:19, 80.5MB/s{'info': ''}]
  3% 49.0M/1.51G [00:00<00:13, 113MB/s{'info': ''}] 
  5% 70.0M/1.51G [00:00<00:10, 142MB/s{'info': ''}]
  6% 89.0M/1.51G [00:00<00:09, 159MB/s{'info': ''}]
  6% 90.0M/1.51G [00:01<00:19, 77.4MB/s{'info': ''}]
  6% 92.0M/1.51G [00:01<00:27, 56.4MB/s{'info': ''}]
  6% 95.0M/1.51G [00:01<00:31, 48.9MB/s{'info': ''}]
  7% 107M/1.51G [00:01<00:22, 66.0MB/s{'info': ''}] 
  8% 127M/1.51G [00:01<00:14, 101MB/s{'info': ''}] 
 10% 162M/1.51G [00:01<00:08, 169MB/s{'info': ''}]
 13% 197M/1.51G [00:01<00:06, 221MB/s{'info': ''}]
 14% 218M/1.51G [00:01<00:06, 

In [18]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import datasets
from datasets import Dataset
from torch.utils.data import DataLoader
from tqdm import trange
from scipy import stats
from transformers import BertModel, BertPreTrainedModel, AutoConfig, AutoTokenizer

## Model Setup

In [19]:
# index map, which used to indicate annotated abstract-title pairs.
map80 = [[3, 2, 4, 6, 0, 5],
 [1, 5, 6, 3, 0, 4],
 [3, 2, 6, 1, 0, 4],
 [2, 1, 6, 5, 4, 0],
 [0, 3, 6, 5, 2, 4],
 [4, 3, 1, 2, 0, 6],
 [6, 1, 0, 4, 5, 3],
 [6, 0, 5, 1, 3, 2],
 [0, 3, 4, 1, 2, 5],
 [0, 3, 2, 6, 5, 4],
 [4, 3, 0, 5, 1, 6],
 [0, 1, 4, 6, 5, 2],
 [4, 0, 5, 1, 2, 6],
 [6, 2, 5, 0, 3, 1],
 [0, 1, 6, 4, 2, 5],
 [4, 6, 0, 3, 1, 5],
 [3, 1, 5, 4, 0, 2],
 [3, 5, 0, 2, 1, 6],
 [0, 6, 1, 2, 4, 3],
 [2, 4, 5, 1, 0, 6],
 [5, 3, 1, 2, 0, 4],
 [3, 6, 2, 4, 5, 0],
 [0, 5, 6, 3, 1, 2],
 [0, 3, 5, 6, 1, 4],
 [0, 4, 2, 5, 3, 1],
 [2, 3, 4, 0, 5, 1],
 [1, 2, 4, 0, 5, 6],
 [3, 5, 4, 1, 6, 0],
 [4, 6, 0, 1, 5, 2],
 [5, 0, 3, 1, 4, 6],
 [3, 6, 2, 5, 4, 0],
 [2, 3, 6, 5, 0, 4],
 [4, 0, 6, 3, 5, 1],
 [0, 2, 3, 5, 1, 6],
 [2, 1, 5, 0, 6, 3],
 [0, 6, 5, 1, 4, 2],
 [4, 5, 0, 2, 6, 1],
 [4, 2, 5, 6, 3, 0],
 [5, 3, 2, 4, 0, 1],
 [3, 5, 4, 2, 6, 0],
 [1, 2, 4, 3, 5, 0],
 [2, 3, 6, 1, 4, 0],
 [6, 3, 4, 0, 1, 2],
 [4, 0, 5, 1, 2, 3],
 [3, 4, 1, 0, 2, 6],
 [4, 0, 1, 3, 6, 2],
 [0, 5, 4, 6, 2, 3],
 [0, 1, 2, 6, 5, 3],
 [5, 3, 0, 4, 1, 6],
 [5, 4, 1, 0, 2, 6],
 [6, 4, 1, 5, 3, 0],
 [4, 0, 5, 3, 2, 6],
 [4, 0, 1, 5, 6, 2],
 [6, 0, 4, 5, 3, 2],
 [6, 0, 2, 1, 4, 3],
 [3, 2, 4, 1, 0, 5],
 [4, 2, 0, 5, 1, 3],
 [0, 2, 6, 3, 5, 4],
 [4, 1, 6, 0, 2, 5],
 [6, 4, 2, 1, 3, 0],
 [2, 5, 3, 4, 0, 1],
 [6, 4, 1, 0, 3, 5],
 [0, 2, 6, 1, 4, 5],
 [6, 5, 4, 3, 0, 2],
 [3, 2, 4, 6, 5, 0],
 [2, 1, 4, 3, 0, 5],
 [1, 6, 2, 3, 0, 5],
 [2, 5, 0, 6, 4, 3],
 [3, 2, 0, 4, 1, 6],
 [0, 3, 2, 4, 6, 1],
 [0, 5, 2, 3, 1, 4],
 [6, 4, 0, 2, 5, 1],
 [6, 0, 3, 5, 1, 4],
 [0, 5, 3, 4, 2, 6],
 [5, 3, 2, 0, 4, 1],
 [1, 4, 3, 5, 0, 6],
 [6, 2, 5, 3, 0, 4],
 [0, 5, 3, 6, 2, 1],
 [2, 0, 4, 6, 5, 1],
 [0, 4, 2, 1, 3, 5]]

selected_index = [153, 154, 156, 159, 161, 164, 165, 167, 168, 172, 174 ,175, 176, 177, 180, 185, 186, 189, 191, 197, 206, 207, 208, 211, 216, 218, 221, 223, 225, 227, 228, 238, 239, 241, 243, 244, 248, 250, 259, 260, 262, 269, 270, 271 ,275, 287, 288, 291, 294, 299]

map50= [[1, 0, 3, 4, 6, 5],
 [2, 1, 3, 0, 4, 6],
 [2, 3, 4, 1, 6, 0],
 [1, 0, 6, 5, 2, 3],
 [0, 1, 5, 3, 4, 6],
 [3, 2, 1, 5, 0, 4],
 [1, 0, 6, 5, 4, 2],
 [0, 4, 2, 6, 1, 3],
 [0, 3, 6, 1, 2, 5],
 [6, 2, 1, 4, 0, 3],
 [5, 0, 4, 2, 6, 3],
 [1, 0, 4, 3, 5, 2],
 [4, 0, 5, 6, 2, 3],
 [6, 1, 0, 4, 3, 5],
 [5, 6, 0, 3, 1, 2],
 [2, 5, 3, 4, 0, 6],
 [0, 2, 6, 5, 4, 1],
 [0, 3, 2, 1, 4, 6],
 [2, 5, 1, 6, 0, 4],
 [2, 5, 3, 1, 4, 0],
 [3, 4, 1, 6, 2, 0],
 [6, 3, 2, 1, 0, 5],
 [0, 6, 5, 1, 2, 4],
 [0, 6, 1, 3, 5, 4],
 [3, 2, 5, 1, 4, 0],
 [3, 4, 0, 5, 1, 6],
 [2, 5, 4, 0, 6, 1],
 [4, 2, 1, 6, 0, 3],
 [2, 4, 5, 6, 0, 3],
 [0, 5, 6, 2, 3, 1],
 [0, 5, 4, 3, 1, 2],
 [4, 1, 6, 5, 2, 0],
 [5, 3, 1, 0, 2, 6],
 [1, 5, 2, 4, 3, 0],
 [2, 1, 0, 3, 4, 5],
 [2, 0, 4, 5, 6, 3],
 [2, 5, 4, 1, 6, 0],
 [0, 5, 2, 1, 6, 4],
 [0, 5, 2, 4, 3, 1],
 [5, 2, 6, 1, 0, 3],
 [0, 2, 3, 6, 4, 5],
 [4, 5, 6, 2, 3, 0],
 [6, 2, 5, 4, 1, 0],
 [2, 6, 0, 1, 3, 5],
 [0, 1, 4, 5, 6, 2],
 [5, 1, 4, 6, 0, 3],
 [6, 0, 5, 4, 2, 1],
 [6, 4, 1, 0, 3, 2],
 [5, 0, 2, 3, 4, 6],
 [2, 3, 0, 1, 6, 5]]

map_model = [['bart_xsum', 'bart_cnn', 't5', 'pegasus_xsum', 'original', 'gpt2'],
 ['bart_base', 'gpt2', 'pegasus_xsum', 'bart_xsum', 'original', 't5'],
 ['bart_xsum', 'bart_cnn', 'pegasus_xsum', 'bart_base', 'original', 't5'],
 ['bart_cnn', 'bart_base', 'pegasus_xsum', 'gpt2', 't5', 'original'],
 ['original', 'bart_xsum', 'pegasus_xsum', 'gpt2', 'bart_cnn', 't5'],
 ['t5', 'bart_xsum', 'bart_base', 'bart_cnn', 'original', 'pegasus_xsum'],
 ['pegasus_xsum', 'bart_base', 'original', 't5', 'gpt2', 'bart_xsum'],
 ['pegasus_xsum', 'original', 'gpt2', 'bart_base', 'bart_xsum', 'bart_cnn'],
 ['original', 'bart_xsum', 't5', 'bart_base', 'bart_cnn', 'gpt2'],
 ['original', 'bart_xsum', 'bart_cnn', 'pegasus_xsum', 'gpt2', 't5'],
 ['t5', 'bart_xsum', 'original', 'gpt2', 'bart_base', 'pegasus_xsum'],
 ['original', 'bart_base', 't5', 'pegasus_xsum', 'gpt2', 'bart_cnn'],
 ['t5', 'original', 'gpt2', 'bart_base', 'bart_cnn', 'pegasus_xsum'],
 ['pegasus_xsum', 'bart_cnn', 'gpt2', 'original', 'bart_xsum', 'bart_base'],
 ['original', 'bart_base', 'pegasus_xsum', 't5', 'bart_cnn', 'gpt2'],
 ['t5', 'pegasus_xsum', 'original', 'bart_xsum', 'bart_base', 'gpt2'],
 ['bart_xsum', 'bart_base', 'gpt2', 't5', 'original', 'bart_cnn'],
 ['bart_xsum', 'gpt2', 'original', 'bart_cnn', 'bart_base', 'pegasus_xsum'],
 ['original', 'pegasus_xsum', 'bart_base', 'bart_cnn', 't5', 'bart_xsum'],
 ['bart_cnn', 't5', 'gpt2', 'bart_base', 'original', 'pegasus_xsum'],
 ['gpt2', 'bart_xsum', 'bart_base', 'bart_cnn', 'original', 't5'],
 ['bart_xsum', 'pegasus_xsum', 'bart_cnn', 't5', 'gpt2', 'original'],
 ['original', 'gpt2', 'pegasus_xsum', 'bart_xsum', 'bart_base', 'bart_cnn'],
 ['original', 'bart_xsum', 'gpt2', 'pegasus_xsum', 'bart_base', 't5'],
 ['original', 't5', 'bart_cnn', 'gpt2', 'bart_xsum', 'bart_base'],
 ['bart_cnn', 'bart_xsum', 't5', 'original', 'gpt2', 'bart_base'],
 ['bart_base', 'bart_cnn', 't5', 'original', 'gpt2', 'pegasus_xsum'],
 ['bart_xsum', 'gpt2', 't5', 'bart_base', 'pegasus_xsum', 'original'],
 ['t5', 'pegasus_xsum', 'original', 'bart_base', 'gpt2', 'bart_cnn'],
 ['gpt2', 'original', 'bart_xsum', 'bart_base', 't5', 'pegasus_xsum'],
 ['bart_xsum', 'pegasus_xsum', 'bart_cnn', 'gpt2', 't5', 'original'],
 ['bart_cnn', 'bart_xsum', 'pegasus_xsum', 'gpt2', 'original', 't5'],
 ['t5', 'original', 'pegasus_xsum', 'bart_xsum', 'gpt2', 'bart_base'],
 ['original', 'bart_cnn', 'bart_xsum', 'gpt2', 'bart_base', 'pegasus_xsum'],
 ['bart_cnn', 'bart_base', 'gpt2', 'original', 'pegasus_xsum', 'bart_xsum'],
 ['original', 'pegasus_xsum', 'gpt2', 'bart_base', 't5', 'bart_cnn'],
 ['t5', 'gpt2', 'original', 'bart_cnn', 'pegasus_xsum', 'bart_base'],
 ['t5', 'bart_cnn', 'gpt2', 'pegasus_xsum', 'bart_xsum', 'original'],
 ['gpt2', 'bart_xsum', 'bart_cnn', 't5', 'original', 'bart_base'],
 ['bart_xsum', 'gpt2', 't5', 'bart_cnn', 'pegasus_xsum', 'original'],
 ['bart_base', 'bart_cnn', 't5', 'bart_xsum', 'gpt2', 'original'],
 ['bart_cnn', 'bart_xsum', 'pegasus_xsum', 'bart_base', 't5', 'original'],
 ['pegasus_xsum', 'bart_xsum', 't5', 'original', 'bart_base', 'bart_cnn'],
 ['t5', 'original', 'gpt2', 'bart_base', 'bart_cnn', 'bart_xsum'],
 ['bart_xsum', 't5', 'bart_base', 'original', 'bart_cnn', 'pegasus_xsum'],
 ['t5', 'original', 'bart_base', 'bart_xsum', 'pegasus_xsum', 'bart_cnn'],
 ['original', 'gpt2', 't5', 'pegasus_xsum', 'bart_cnn', 'bart_xsum'],
 ['original', 'bart_base', 'bart_cnn', 'pegasus_xsum', 'gpt2', 'bart_xsum'],
 ['gpt2', 'bart_xsum', 'original', 't5', 'bart_base', 'pegasus_xsum'],
 ['gpt2', 't5', 'bart_base', 'original', 'bart_cnn', 'pegasus_xsum'],
 ['bart_base', 'original', 'bart_xsum', 't5', 'pegasus_xsum', 'gpt2'],
 ['bart_cnn', 'bart_base', 'bart_xsum', 'original', 't5', 'pegasus_xsum'],
 ['bart_cnn', 'bart_xsum', 't5', 'bart_base', 'pegasus_xsum', 'original'],
 ['bart_base', 'original', 'pegasus_xsum', 'gpt2', 'bart_cnn', 'bart_xsum'],
 ['original', 'bart_base', 'gpt2', 'bart_xsum', 't5', 'pegasus_xsum'],
 ['bart_xsum', 'bart_cnn', 'bart_base', 'gpt2', 'original', 't5'],
 ['bart_base', 'original', 'pegasus_xsum', 'gpt2', 't5', 'bart_cnn'],
 ['original', 't5', 'bart_cnn', 'pegasus_xsum', 'bart_base', 'bart_xsum'],
 ['original', 'bart_xsum', 'pegasus_xsum', 'bart_base', 'bart_cnn', 'gpt2'],
 ['pegasus_xsum', 'bart_cnn', 'bart_base', 't5', 'original', 'bart_xsum'],
 ['gpt2', 'original', 't5', 'bart_cnn', 'pegasus_xsum', 'bart_xsum'],
 ['bart_base', 'original', 't5', 'bart_xsum', 'gpt2', 'bart_cnn'],
 ['t5', 'original', 'gpt2', 'pegasus_xsum', 'bart_cnn', 'bart_xsum'],
 ['pegasus_xsum', 'bart_base', 'original', 't5', 'bart_xsum', 'gpt2'],
 ['gpt2', 'pegasus_xsum', 'original', 'bart_xsum', 'bart_base', 'bart_cnn'],
 ['bart_cnn', 'gpt2', 'bart_xsum', 't5', 'original', 'pegasus_xsum'],
 ['original', 'bart_cnn', 'pegasus_xsum', 'gpt2', 't5', 'bart_base'],
 ['original', 'bart_xsum', 'bart_cnn', 'bart_base', 't5', 'pegasus_xsum'],
 ['bart_cnn', 'gpt2', 'bart_base', 'pegasus_xsum', 'original', 't5'],
 ['bart_cnn', 'gpt2', 'bart_xsum', 'bart_base', 't5', 'original'],
 ['bart_xsum', 't5', 'bart_base', 'pegasus_xsum', 'bart_cnn', 'original'],
 ['pegasus_xsum', 'bart_xsum', 'bart_cnn', 'bart_base', 'original', 'gpt2'],
 ['original', 'pegasus_xsum', 'gpt2', 'bart_base', 'bart_cnn', 't5'],
 ['original', 'pegasus_xsum', 'bart_base', 'bart_xsum', 'gpt2', 't5'],
 ['bart_xsum', 'bart_cnn', 'gpt2', 'bart_base', 't5', 'original'],
 ['bart_xsum', 't5', 'original', 'gpt2', 'bart_base', 'pegasus_xsum'],
 ['bart_cnn', 'gpt2', 't5', 'original', 'pegasus_xsum', 'bart_base'],
 ['t5', 'bart_cnn', 'bart_base', 'pegasus_xsum', 'original', 'bart_xsum'],
 ['bart_cnn', 't5', 'gpt2', 'pegasus_xsum', 'original', 'bart_xsum'],
 ['original', 'gpt2', 'pegasus_xsum', 'bart_cnn', 'bart_xsum', 'bart_base'],
 ['original', 'gpt2', 't5', 'bart_xsum', 'bart_base', 'bart_cnn'],
 ['t5', 'bart_base', 'pegasus_xsum', 'gpt2', 'bart_cnn', 'original'],
 ['gpt2', 'bart_xsum', 'bart_base', 'original', 'bart_cnn', 'pegasus_xsum'],
 ['bart_base', 'gpt2', 'bart_cnn', 't5', 'bart_xsum', 'original'],
 ['bart_cnn', 'bart_base', 'original', 'bart_xsum', 't5', 'gpt2'],
 ['bart_cnn', 'original', 't5', 'gpt2', 'pegasus_xsum', 'bart_xsum'],
 ['bart_cnn', 'gpt2', 't5', 'bart_base', 'pegasus_xsum', 'original'],
 ['original', 'gpt2', 'bart_cnn', 'bart_base', 'pegasus_xsum', 't5'],
 ['original', 'gpt2', 'bart_cnn', 't5', 'bart_xsum', 'bart_base'],
 ['gpt2', 'bart_cnn', 'pegasus_xsum', 'bart_base', 'original', 'bart_xsum'],
 ['original', 'bart_cnn', 'bart_xsum', 'pegasus_xsum', 't5', 'gpt2'],
 ['t5', 'gpt2', 'pegasus_xsum', 'bart_cnn', 'bart_xsum', 'original'],
 ['pegasus_xsum', 'bart_cnn', 'gpt2', 't5', 'bart_base', 'original'],
 ['bart_cnn', 'pegasus_xsum', 'original', 'bart_base', 'bart_xsum', 'gpt2'],
 ['original', 'bart_base', 't5', 'gpt2', 'pegasus_xsum', 'bart_cnn'],
 ['gpt2', 'bart_base', 't5', 'pegasus_xsum', 'original', 'bart_xsum'],
 ['pegasus_xsum', 'original', 'gpt2', 't5', 'bart_cnn', 'bart_base'],
 ['pegasus_xsum', 't5', 'bart_base', 'original', 'bart_xsum', 'bart_cnn'],
 ['gpt2', 'original', 'bart_cnn', 'bart_xsum', 't5', 'pegasus_xsum'],
 ['bart_cnn', 'bart_xsum', 'original', 'bart_base', 'pegasus_xsum', 'gpt2'],
 ['bart_base', 'original', 'bart_xsum', 't5', 'pegasus_xsum', 'gpt2'],
 ['bart_cnn', 'bart_base', 'bart_xsum', 'original', 't5', 'pegasus_xsum'],
 ['bart_cnn', 'bart_xsum', 't5', 'bart_base', 'pegasus_xsum', 'original'],
 ['bart_base', 'original', 'pegasus_xsum', 'gpt2', 'bart_cnn', 'bart_xsum'],
 ['original', 'bart_base', 'gpt2', 'bart_xsum', 't5', 'pegasus_xsum'],
 ['bart_xsum', 'bart_cnn', 'bart_base', 'gpt2', 'original', 't5'],
 ['bart_base', 'original', 'pegasus_xsum', 'gpt2', 't5', 'bart_cnn'],
 ['original', 't5', 'bart_cnn', 'pegasus_xsum', 'bart_base', 'bart_xsum'],
 ['original', 'bart_xsum', 'pegasus_xsum', 'bart_base', 'bart_cnn', 'gpt2'],
 ['pegasus_xsum', 'bart_cnn', 'bart_base', 't5', 'original', 'bart_xsum'],
 ['gpt2', 'original', 't5', 'bart_cnn', 'pegasus_xsum', 'bart_xsum'],
 ['bart_base', 'original', 't5', 'bart_xsum', 'gpt2', 'bart_cnn'],
 ['t5', 'original', 'gpt2', 'pegasus_xsum', 'bart_cnn', 'bart_xsum'],
 ['pegasus_xsum', 'bart_base', 'original', 't5', 'bart_xsum', 'gpt2'],
 ['gpt2', 'pegasus_xsum', 'original', 'bart_xsum', 'bart_base', 'bart_cnn'],
 ['bart_cnn', 'gpt2', 'bart_xsum', 't5', 'original', 'pegasus_xsum'],
 ['original', 'bart_cnn', 'pegasus_xsum', 'gpt2', 't5', 'bart_base'],
 ['original', 'bart_xsum', 'bart_cnn', 'bart_base', 't5', 'pegasus_xsum'],
 ['bart_cnn', 'gpt2', 'bart_base', 'pegasus_xsum', 'original', 't5'],
 ['bart_cnn', 'gpt2', 'bart_xsum', 'bart_base', 't5', 'original'],
 ['bart_xsum', 't5', 'bart_base', 'pegasus_xsum', 'bart_cnn', 'original'],
 ['pegasus_xsum', 'bart_xsum', 'bart_cnn', 'bart_base', 'original', 'gpt2'],
 ['original', 'pegasus_xsum', 'gpt2', 'bart_base', 'bart_cnn', 't5'],
 ['original', 'pegasus_xsum', 'bart_base', 'bart_xsum', 'gpt2', 't5'],
 ['bart_xsum', 'bart_cnn', 'gpt2', 'bart_base', 't5', 'original'],
 ['bart_xsum', 't5', 'original', 'gpt2', 'bart_base', 'pegasus_xsum'],
 ['bart_cnn', 'gpt2', 't5', 'original', 'pegasus_xsum', 'bart_base'],
 ['t5', 'bart_cnn', 'bart_base', 'pegasus_xsum', 'original', 'bart_xsum'],
 ['bart_cnn', 't5', 'gpt2', 'pegasus_xsum', 'original', 'bart_xsum'],
 ['original', 'gpt2', 'pegasus_xsum', 'bart_cnn', 'bart_xsum', 'bart_base'],
 ['original', 'gpt2', 't5', 'bart_xsum', 'bart_base', 'bart_cnn'],
 ['t5', 'bart_base', 'pegasus_xsum', 'gpt2', 'bart_cnn', 'original'],
 ['gpt2', 'bart_xsum', 'bart_base', 'original', 'bart_cnn', 'pegasus_xsum'],
 ['bart_base', 'gpt2', 'bart_cnn', 't5', 'bart_xsum', 'original'],
 ['bart_cnn', 'bart_base', 'original', 'bart_xsum', 't5', 'gpt2'],
 ['bart_cnn', 'original', 't5', 'gpt2', 'pegasus_xsum', 'bart_xsum'],
 ['bart_cnn', 'gpt2', 't5', 'bart_base', 'pegasus_xsum', 'original'],
 ['original', 'gpt2', 'bart_cnn', 'bart_base', 'pegasus_xsum', 't5'],
 ['original', 'gpt2', 'bart_cnn', 't5', 'bart_xsum', 'bart_base'],
 ['gpt2', 'bart_cnn', 'pegasus_xsum', 'bart_base', 'original', 'bart_xsum'],
 ['original', 'bart_cnn', 'bart_xsum', 'pegasus_xsum', 't5', 'gpt2'],
 ['t5', 'gpt2', 'pegasus_xsum', 'bart_cnn', 'bart_xsum', 'original'],
 ['pegasus_xsum', 'bart_cnn', 'gpt2', 't5', 'bart_base', 'original'],
 ['bart_cnn', 'pegasus_xsum', 'original', 'bart_base', 'bart_xsum', 'gpt2'],
 ['original', 'bart_base', 't5', 'gpt2', 'pegasus_xsum', 'bart_cnn'],
 ['gpt2', 'bart_base', 't5', 'pegasus_xsum', 'original', 'bart_xsum'],
 ['pegasus_xsum', 'original', 'gpt2', 't5', 'bart_cnn', 'bart_base'],
 ['pegasus_xsum', 't5', 'bart_base', 'original', 'bart_xsum', 'bart_cnn'],
 ['gpt2', 'original', 'bart_cnn', 'bart_xsum', 't5', 'pegasus_xsum'],
 ['bart_cnn', 'bart_xsum', 'original', 'bart_base', 'pegasus_xsum', 'gpt2']]


def displaysmaples(samples):
  i = 0
  titles = samples.title.to_list()
  abstract = samples.abstract.to_list()
  bart_base = samples.bart_base.to_list()
  bart_cnn = samples.bart_cnn.to_list()
  bart_xsum = samples.bart_xsum.to_list()
  t5_small = samples.t5_small.to_list()
  gpt2 = samples.gpt2.to_list()
  pegasus_xsum = samples.pegasus_xsum.to_list()

  for t, a, t1, t2, t3, t4, t5, t6 in zip(titles, abstract, bart_base, bart_cnn, bart_xsum, t5_small, gpt2, pegasus_xsum):
    print("original title: ", t)
    print("abstract: ", a)
    print("from bart_base: ")
    gt1 = t1.split("<TITLE>")[1:]
    for gt in gt1:
      print(gt)
    print("\n")
    print("from bart_cnn: ")
    gt1 = t2.split("<TITLE>")[1:]
    for gt in gt1:
      print(gt)
    print("\n")
    print("from bart_xsum: ")
    gt1 = t3.split("<TITLE>")[1:]
    for gt in gt1:
      print(gt)
    print("\n")
    print("from t5_small: ")
    gt1 = t4.split("<TITLE>")[1:]
    for gt in gt1:
      print(gt)
    print("\n")
    print("from gpt2: ")
    gt1 = t5.split("<TITLE>")[1:]
    for gt in gt1:
      print(gt)
    print("\n")
    print("from pegasus_xsum: ")
    gt1 = t6.split("<TITLE>")[1:]
    for gt in gt1:
      print(gt)
    print("\n")
    i = i + 1

    if i >= 20:
      break

# index von Optionen der human evaluation, 20 - 99 samples und 100-150(von index 0 bis 49)
# [4, 3, 5, 2, 0, 1] bedeutet
#1.op aus t5_samall,
#2.op aus bart_xsum,
#3.op aus gpt2,
#4.op aus bart_cnn,
#5.op original,
#6.op aus bart_base,

def getindex():
  result = []
  for i in range(80):
    col = np.arange(1,7)
    np.random.shuffle(col)
    col = col.tolist()[:-1]
    col.append(0)
    np.random.shuffle(col)
    result.append(col)
  return result


def sampler(samples, map20, start, end):
  i = start
  titles = samples.title.to_list()[start: end]
  abstract = samples.abstract.to_list()[start: end]
  bart_base = samples.bart_base.to_list()[start: end]
  bart_cnn = samples.bart_cnn.to_list()[start: end]
  bart_xsum = samples.bart_xsum.to_list()[start: end]
  t5_small = samples.t5_small.to_list()[start: end]
  gpt2 = samples.gpt2.to_list()[start: end]
  pegasus_xsum = samples.pegasus_xsum.to_list()[start: end]

  tmap = []
  for t, a, t1, t2, t3, t4, t5, t6 in zip(titles, abstract, bart_base, bart_cnn, bart_xsum, t5_small, gpt2, pegasus_xsum):
    gt1 = t1.split("<TITLE>")[1:][0]
    gt2 = t2.split("<TITLE>")[1:][0]
    gt3 = t3.split("<TITLE>")[1:][0]
    gt4 = t4.split("<TITLE>")[1:][0]
    gt5 = t5.split("<TITLE>")[1:][0]
    gt6 = t6.split("<TITLE>")[1:][0]
    col = [t, gt1, gt2, gt3, gt4, gt5, gt6]
    rcol = [col[map20[i][j]] for j in range(6)]
    rcol.append(a)
    tmap.append(rcol)
    i = i + 1

  return tmap
def map2model(ls):
  r = []
  for i in ls:
    if i == 0:
      r.append("original")
    if i == 1:
      r.append("bart_base")
    if i == 2:
      r.append("bart_cnn")
    if i == 3:
      r.append("bart_xsum")
    if i == 4:
      r.append("t5")
    if i == 5:
      r.append("gpt2")
    if i == 6:
      r.append("pegasus_xsum")
  return r

def map2index(ls):
  r = []
  for i in ls:
    if i == "original":
      r.append(0)
    if i == "bart_base":
      r.append(1)
    if i == "bart_cnn":
      r.append(2)
    if i == "bart_xsum":
      r.append(3)
    if i == "t5":
      r.append(4)
    if i == "gpt2":
      r.append(5)
    if i == "pegasus_xsum":
      r.append(6)
  return r

def map2modelAM(ls):
  r = []
  for i in ls:
    if i == 0:
      r.append("bart_base")
    if i == 1:
      r.append("bart_cnn")
    if i == 2:
      r.append("bart_xsum")
    if i == 3:
      r.append("t5")
    if i == 4:
      r.append("gpt2")
    if i == 5:
      r.append("pegasus_xsum")
  return r


# index von Optionen der human evaluation, 150 - 200 samples
# [4, 3, 5, 2, 0, 1] bedeutet
#1.op aus t5_samall,
#2.op aus bart_xsum,
#3.op aus gpt2,
#4.op aus bart_cnn,
#5.op original,
#6.op aus bart_base,

def getindex():
  result = []
  for i in range(50):
    col = np.arange(1,7)
    np.random.shuffle(col)
    col = col.tolist()[:-1]
    col.append(0)
    np.random.shuffle(col)
    result.append(col)
  return result

#map für Teinehmer

def getTmap():
  i = 0
  b = False
  map30 = []
  counters = np.zeros(20).tolist()
  while not b:
    personMap = np.arange(20)
    np.random.shuffle(personMap)
    personMap = personMap[:10]
    i0 = personMap[0]
    i1 = personMap[1]

    if (counters[i0] < 3) and (counters[i1] < 3):
      counters[i0] = counters[i0] + 1
      counters[i1] = counters[i1] + 1
      map30.append(personMap)
    tb = True
    for t in counters:
      tb = tb and (t == 3)
    b = tb
    if b == True:
      print("terminiert at:", counters)
  return map30

---

# **Fine Tuning**


---


In [21]:
class Excerpt_Dataset(Dataset):

    def __init__(self, data, maxlen, tokenizer):
        #Store the contents of the file in a pandas dataframe
        self.df = data.reset_index()
        #Initialize the tokenizer for the desired transformer model
        self.tokenizer = tokenizer
        #Maximum length of the tokens list to keep all the sequences of fixed size
        self.maxlen = maxlen

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        #Select the sentence and label at the specified index in the data frame
        excerpt = self.df.loc[index, 'excerpt']
        try:
            target = self.df.loc[index, 'target']
        except:
            target = 0.0
        #identifier = self.df.loc[index, 'id']
        #Preprocess the text to be suitable for the transformer
        tokens = self.tokenizer.tokenize(excerpt)
        tokens = ['[CLS]'] + tokens + ['[SEP]']
        if len(tokens) < self.maxlen:
            tokens = tokens + ['[PAD]' for _ in range(self.maxlen - len(tokens))]
        else:
            tokens = tokens[:self.maxlen-1] + ['[SEP]']
        #Obtain the indices of the tokens in the BERT Vocabulary
        input_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        input_ids = torch.tensor(input_ids)
        #Obtain the attention mask i.e a tensor containing 1s for no padded tokens and 0s for padded ones
        attention_mask = (input_ids != 0).long()

        target = torch.tensor([target], dtype=torch.float32)

        return input_ids, attention_mask, target

In [22]:
'''
class BertRegresser(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)
        #The output layer that takes the [CLS] representation and gives an output
        self.cls_layer1 = nn.Linear(config.hidden_size,128)
        self.relu1 = nn.ReLU()
        self.ff1 = nn.Linear(128,128)
        self.tanh1 = nn.Tanh()
        self.ff2 = nn.Linear(128,1)

    def forward(self, input_ids, attention_mask):
        #Feed the input to Bert model to obtain contextualized representations
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        #Obtain the representations of [CLS] heads
        logits = outputs.last_hidden_state[:,0,:]
        output = self.cls_layer1(logits)
        output = self.relu1(output)
        output = self.ff1(output)
        output = self.tanh1(output)
        output = self.ff2(output)
        return output
'''
class BertRegresser(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)
        #The output layer that takes the [CLS] representation and gives an output
        self.regressor = nn.Sequential(
            nn.Dropout(p['dropout']),
            nn.Linear(768, 1))

    def forward(self, input_ids, attention_mask):
        #Feed the input to Bert model to obtain contextualized representations
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        #Obtain the representations of [CLS] heads
        logits = outputs[1]
        output = self.regressor(logits)

        return output



## Evaluation

In [40]:
def setup_seed(seed):
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  np.random.seed(seed)
  torch.backends.cudnn.deteministic = True

## Model Configurations
p = {
    'max_len': 512,
    'batch_size': 6,
    'lr' : 4.063769241800226e-05,
    'epochs': 12,
    'dropout': 0.5,
    'num_threads' : 1,
    #'model_name' : 'allenai/scibert_scivocab_uncased',
    'model_name' : 'bert-base-uncased',
    'do_train' : True,
    'random_seed': 24
}

setup_seed(p['random_seed'])

In [41]:
def evaluate(model, criterion, dataloader, device):
    model.eval()
    mean_acc, mean_loss, count = 0, 0, 0
    preds = []
    lst_label = []
    with torch.no_grad():
        for input_ids, attention_mask, target in (dataloader):

            input_ids, attention_mask, target = input_ids.to(device), attention_mask.to(device), target.to(device)
            output = model(input_ids, attention_mask)
            preds += output
            lst_label += target
            mean_loss += criterion(output, target.type_as(output)).item()
#             mean_err += get_rmse(output, target)
            count += 1
        predss = np.array([x.cpu().data.numpy().tolist() for x in preds]).squeeze()
        lst_labels = np.array([x.cpu().data.numpy().tolist() for x in lst_label]).squeeze()
        corr = stats.spearmanr(predss, lst_labels)
    return corr[0] #mean_loss/count

def train(model, criterion, optimizer, train_loader, val_loader, epochs, device):
    best_acc = 0
    for epoch in trange(epochs, desc="Epoch"):
        model.train()
        train_loss = 0
        for i, (input_ids, attention_mask, target) in enumerate(iterable=train_loader):
            optimizer.zero_grad()
            input_ids, attention_mask, target = input_ids.to(device), attention_mask.to(device), target.to(device)

            output = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = criterion(output, target.type_as(output))
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        print(f"Training loss is {train_loss/len(train_loader)}")
        val_loss = evaluate(model=model, criterion=criterion, dataloader=val_loader, device=device)
        print("Epoch {} complete! Correlations : {}".format(epoch, val_loss))


## Configuration loaded from AutoConfig
aconfig = AutoConfig.from_pretrained(p['model_name'])
## Tokenizer loaded from AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(p['model_name'])
## Creating the model from the desired transformer model
model = BertRegresser.from_pretrained(p['model_name'], config=aconfig)


#frezze all layers except regression head
'''
unfreeze_layers  = ['bert.pooler', 'cls_layer1.', 'ff1.', 'ff2']

for name, params in model.named_parameters():
  params.requires_grad = False
  for ele in unfreeze_layers:
    if ele in name:
      params.requires_grad = True
      break

for name, params in model.named_parameters():
  if params.requires_grad:
    print(name, params.size())
'''

unfreeze_layers = ['bert.pooler', 'regressor.1']
for name, params in model.named_parameters():
  params.requires_grad = False
  for ele in unfreeze_layers:
    if ele in name:
      params.requires_grad = True
      break

for name, params in model.named_parameters():
  if params.requires_grad:
    print(name, params.size())

## GPU or CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
## Putting model to device
model = model.to(device)
## Takes as the input the logits of the positive class and computes the binary cross-entropy
# criterion = nn.BCEWithLogitsLoss()
criterion = nn.MSELoss()
## Optimizer
optimizer = optim.Adam(params=model.parameters(), lr=p['lr'])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertRegresser: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertRegresser from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertRegresser from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertRegresser were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['regressor.1.weig

bert.pooler.dense.weight torch.Size([768, 768])
bert.pooler.dense.bias torch.Size([768])
regressor.1.weight torch.Size([1, 768])
regressor.1.bias torch.Size([1])


In [45]:
MODEL_OUT_DIR = f'{PROJECT_ROOT}/output/sciBert_Kaggle'

#for s in p.values():
#  MODEL_OUT_DIR += '_' + str(s).replace('/', '_', 1)

#print(MODEL_OUT_DIR)

/content/drive/MyDrive/DL4NLP/abstract-to-title-generation/output/sciBert_Kaggle


In [43]:
DATA_DIR = f'{PROJECT_ROOT}/data/annotated'
text_map = pd.read_csv(f'{DATA_DIR}/140_annotations_pairs.csv', index_col=0)
scores = pd.read_csv(f'{DATA_DIR}/140_humanannotation_withoutunannotated.csv', index_col=0)

abstract_df =text_map[['abstract']]
title_df = text_map[['title', 'bart_base', 'bart_cnn', 'bart_xsum', 't5_small', 'gpt2', 'pegasus_xsum']]
abstract_np = abstract_df.to_numpy()
scores_np = scores.to_numpy()
title_np = title_df.to_numpy()
idx_map = [map2index(row) for row in map_model[:140]]
title_np_picked = np.array([np.take(row1, np.sort(row2)) for row1, row2 in zip(title_np, idx_map)])
score_np_picked = np.array([np.take(row1, np.sort(row2)) for row1, row2 in zip(scores_np, idx_map)])

pairs_np_picked = np.concatenate([abstract_np, title_np_picked,score_np_picked], axis=1)
pairs_np_picked_shuffled = np.random.permutation(pairs_np_picked)

abstracs = pairs_np_picked_shuffled[:,:1]
titles = pairs_np_picked_shuffled[:,1:7]
scores = pairs_np_picked_shuffled[:,7:].astype(float)
scores = np.around(scores, 4)

lst = []

for ab, row1, row2 in zip(abstracs, titles, scores):
  assert len(row1) == len(row2)
  assert len(row1) == 6
  for t, s in zip(row1, row2):

    if np.isnan(s):
      print('found nan score')
    if t=='':
      print('found empty title')
    lst.append([ab[0] + '[SEP]' + t, s])
df = pd.DataFrame(np.array(lst))

df.columns = ['excerpt', 'target']
#dataframe = dataframe.sample(frac=1, random_state=42).reset_index(drop=True)

dftrain = df[:600].reset_index(drop=True)
dfdev = df[600:660].reset_index(drop=True)
dftest = df[660:].reset_index(drop=True)

OUT_DIR = f'{MODEL_OUT_DIR}/robust_test'

os.makedirs(OUT_DIR, exist_ok=True)

dftrain.to_csv(f'{OUT_DIR}/sciBert_shuffled_train.csv')
dfdev.to_csv(f'{OUT_DIR}/sciBert_shuffled_dev.csv')
dftest.to_csv(f'{OUT_DIR}/sciBert_shuffled_test.csv')

dftrain = pd.read_csv(f'{OUT_DIR}/sciBert_shuffled_train.csv', index_col=0)
dfdev = pd.read_csv(f'{OUT_DIR}/sciBert_shuffled_dev.csv', index_col=0)
dftest = pd.read_csv(f'{OUT_DIR}/sciBert_shuffled_test.csv', index_col=0)


## Training Dataset
train_set = Excerpt_Dataset(data=dftrain, maxlen=p['max_len'], tokenizer=tokenizer)
dev_set = Excerpt_Dataset(data=dfdev, maxlen=p['max_len'], tokenizer=tokenizer)
test_set = Excerpt_Dataset(data=dftest, maxlen=p['max_len'], tokenizer=tokenizer)

## Data Loaders
train_loader = DataLoader(dataset=train_set, batch_size=p['batch_size'], num_workers=p['num_threads'], shuffle=True)
dev_loader = DataLoader(dataset=dev_set, batch_size=p['batch_size'], num_workers=p['num_threads'], shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=p['batch_size'], num_workers=p['num_threads'], shuffle=True)

In [44]:
if p['do_train']:
  train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    train_loader=train_loader,
    val_loader=dev_loader,
    epochs = p['epochs'],
    device = device
  )

Epoch:   0%|          | 0/12 [00:10<?, ?it/s]


KeyboardInterrupt: ignored

In [ ]:

#torch.save(model.state_dict(), MODEL_OUT_DIR + '.model')

#load model
model_state, optimizer_state = torch.load(os.path.join('../output/sciBert_Kaggle/ray_results/runner_2022-04-29_16-44-10/runner_980f0_00002_2_lr=4.0638e-05_2022-04-29_17-04-24/checkpoint_000017', "checkpoint"))
model.load_state_dict(model_state)

#model.load_state_dict(torch.load("../output/sciBert_Kaggle/sciBert.model"))
#model.eval()

In [ ]:
#@title prediction function
def predict(model, dataloader, device):
    predicted_label = []
    actual_label = []
    with torch.no_grad():
        for input_ids, attention_mask, target in (dataloader):
            
            input_ids, attention_mask, target = input_ids.to(device), attention_mask.to(device), target.to(device)
            output = model(input_ids, attention_mask)
                        
            predicted_label += output
            actual_label += target
            
    return predicted_label, actual_label

---

# **Display Correlation**


---


In [ ]:
output,GS_label = predict(model, train_loader, device)
cpu_output = np.array([x.cpu().data.numpy() for x in output]).squeeze()
cpu_target = np.array([x.cpu().data.numpy() for x in GS_label]).squeeze()
stats.spearmanr(cpu_output, cpu_target)[0]

In [ ]:
dev_output,dev_GS_label = predict(model, dev_loader, device)
cpu_dev_output = np.array([x.cpu().data.numpy() for x in dev_output]).squeeze()
cpu_dev_target = np.array([x.cpu().data.numpy() for x in dev_GS_label]).squeeze()
stats.spearmanr(cpu_dev_output, cpu_dev_target)[0]

In [ ]:
test_output,test_GS_label = predict(model, test_loader, device)
cpu_test_output = np.array([x.cpu().data.numpy() for x in test_output]).squeeze()
cpu_test_target = np.array([x.cpu().data.numpy() for x in test_GS_label]).squeeze()
stats.spearmanr(cpu_test_output, cpu_test_target)[0]

In [ ]:
dev_output,dev_GS_label = predict(model, dev_loader, device)
cpu_dev_output = np.array([x.cpu().data.numpy() for x in dev_output]).squeeze()
cpu_dev_target = np.array([x.cpu().data.numpy() for x in dev_GS_label]).squeeze()
stats.spearmanr(cpu_dev_output, cpu_dev_target)[0]

In [ ]:
test_output,test_GS_label = predict(model, test_loader, device)
cpu_test_output = np.array([x.cpu().data.numpy() for x in test_output]).squeeze()
cpu_test_target = np.array([x.cpu().data.numpy() for x in test_GS_label]).squeeze()
stats.spearmanr(cpu_test_output, cpu_test_target)[0]